In [1]:
import gym
import random
import itertools
import numpy as np
from IPython.display import clear_output
from time import sleep
from tqdm import tqdm

def create_random_policy(env):
    policy = {}
    for key in range(0, env.observation_space.n):
        current_end = 0
        p = {}
        for action in range(0, env.action_space.n):
            p[action] = 1 / env.action_space.n
        policy[key] = p
    return policy

def create_state_action_dictionary(env, policy):
    Q = {}
    for key in policy.keys():
        Q[key] = {a: 0.0 for a in range(0, env.action_space.n)}
    return Q

def run_game(env, policy, display=True):
    env.reset()
    episode = []
    finished = False

    while not finished:
        s = env.s
        if display:
            clear_output(True)
            env.render()
            sleep(1)

        timestep = []
        timestep.append(s)
        n = random.uniform(0, sum(policy[s].values()))
        top_range = 0
        for prob in policy[s].items():
            top_range += prob[1]
            if n < top_range:
                action = prob[0]
                break

        state, reward, finished, info = env.step(action)
        timestep.append(action)
        timestep.append(reward)
        episode.append(timestep)

        if display:
            clear_output(True)
            env.render()
            sleep(1)

    return episode

def test_policy(policy, env):
    wins = 0
    r = 100
    for i in range(r):
        w = run_game(env, policy, display=False)[-1][-1]
        if w == 1:
            wins += 1
    return wins / r

def monte_carlo_e_soft(env, episodes=100, policy=None, epsilon=0.01):
    if not policy:
        policy = create_random_policy(env)
    Q = create_state_action_dictionary(env, policy)
    returns = {}

    for _ in tqdm(range(episodes)):
        G = 0
        episode = run_game(env=env, policy=policy, display=False)
        for i in reversed(range(0, len(episode))):
            s_t, a_t, r_t = episode[i]
            state_action = (s_t, a_t)
            G += r_t

            if state_action not in [(x[0], x[1]) for x in episode[0:i]]:
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]

                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action])
                Q_list = list(map(lambda x: x[1], Q[s_t].items()))
                indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]

                max_Q = random.choice(indices)
                A_star = max_Q

                for a in policy[s_t].items():
                    if a[0] == A_star:
                        policy[s_t][a[0]] = 1 - epsilon + (epsilon / abs(sum(policy[s_t].values())))
                    else:
                        policy[s_t][a[0]] = (epsilon / abs(sum(policy[s_t].values())))
                        
import gym
import random
import numpy as np
from IPython.display import clear_output
from time import sleep
from tqdm import tqdm

def create_random_policy(env):
    policy = {}
    for key in range(env.observation_space.n):
        p = {action: 1 / env.action_space.n for action in range(env.action_space.n)}
        policy[key] = p
    return policy

def create_state_action_dictionary(env, policy):
    Q = {}
    for key in policy.keys():
        Q[key] = {a: 0.0 for a in range(env.action_space.n)}
    return Q

def run_game(env, policy, display=False):
    state = env.reset()[0]  # Adjusted for latest gym API
    episode = []
    finished = False
    
    while not finished:
        if display:
            clear_output(True)
            env.render()
            sleep(1)
        
        action_probabilities = policy[state]
        actions, probabilities = zip(*action_probabilities.items())
        action = np.random.choice(actions, p=probabilities)
        
        next_state, reward, finished, _, _ = env.step(action)  # Adjusted for latest gym API
        episode.append((state, action, reward))
        state = next_state
    
    if display:
        clear_output(True)
        env.render()
        sleep(1)
    
    return episode

def test_policy(policy, env, episodes=100):
    wins = 0
    for _ in range(episodes):
        reward = run_game(env, policy, display=False)[-1][-1]
        if reward == 20:  # Reward for successful drop-off in Taxi-v3
            wins += 1
    return wins / episodes

def monte_carlo_e_soft(env, episodes=1000, policy=None, epsilon=0.1):
    if policy is None:
        policy = create_random_policy(env)
    Q = create_state_action_dictionary(env, policy)
    returns = {}
    
    for _ in tqdm(range(episodes)):
        G = 0
        episode = run_game(env, policy, display=False)
        visited_state_actions = set()
        
        for i in reversed(range(len(episode))):
            state, action, reward = episode[i]
            G += reward
            if (state, action) not in visited_state_actions:
                visited_state_actions.add((state, action))
                if (state, action) not in returns:
                    returns[(state, action)] = []
                
                returns[(state, action)].append(G)
                Q[state][action] = np.mean(returns[(state, action)])
                
                best_action = max(Q[state], key=Q[state].get)
                for a in policy[state]:
                    if a == best_action:
                        policy[state][a] = 1 - epsilon + (epsilon / env.action_space.n)
                    else:
                        policy[state][a] = epsilon / env.action_space.n
    
    return policy

# Initialize environment
env = gym.make("Taxi-v3")

# Train policy using Monte Carlo ε-soft policy iteration
policy = monte_carlo_e_soft(env, episodes=10000)
print(policy)

# Test the learned policy
result = test_policy(policy, env)
print(f"Policy success rate: {result * 100:.2f}%")


env = gym.make("Taxi-v3")
policy = monte_carlo_e_soft(env, episodes=1)
print(policy)

result = test_policy(policy, env)
print(result)


  0%|          | 0/10000 [00:00<?, ?it/s]c:\Users\Praveena\myenv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
100%|██████████| 10000/10000 [08:16<00:00, 20.13it/s]


{0: {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}, 1: {0: 0.9166666666666667, 1: 0.016666666666666666, 2: 0.016666666666666666, 3: 0.016666666666666666, 4: 0.016666666666666666, 5: 0.016666666666666666}, 2: {0: 0.016666666666666666, 1: 0.016666666666666666, 2: 0.016666666666666666, 3: 0.016666666666666666, 4: 0.9166666666666667, 5: 0.016666666666666666}, 3: {0: 0.016666666666666666, 1: 0.016666666666666666, 2: 0.016666666666666666, 3: 0.016666666666666666, 4: 0.9166666666666667, 5: 0.016666666666666666}, 4: {0: 0.016666666666666666, 1: 0.016666666666666666, 2: 0.9166666666666667, 3: 0.016666666666666666, 4: 0.016666666666666666, 5: 0.016666666666666666}, 5: {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}, 6: {0: 0.9166666666666667, 1: 0.016666666666666666, 2: 0.016666666666666666, 3: 0.0166666

100%|██████████| 1/1 [00:00<00:00,  5.14it/s]


{0: {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}, 1: {0: 0.016666666666666666, 1: 0.016666666666666666, 2: 0.016666666666666666, 3: 0.016666666666666666, 4: 0.9166666666666667, 5: 0.016666666666666666}, 2: {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}, 3: {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}, 4: {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}, 5: {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}, 6: {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4